In [1]:
source("/home/user/data2/lit/bin/lit_utils.R")
source("/home/user/data3/lit/project/sORFs/sORFs.utils.R")
lib_text()
lib_plot()

In [27]:
orfs_path <- "../../results/1/orfs_merged_final.tsv"
fread_c(orfs_path) -> orfs
head(orfs,1)
colnames(orfs)
count(orfs,Is_canonical)

,ORF_id,ORF_type,Start_codon,Isoform_id,Chr,Strand,ORF_seq,ORF_length,Geneid,Isoform_structural_category,⋯,RPF_codon_coverage,Psites_codon_coverage,FL,FL_TPM,N,C,A,Gene_type,Unique_peptide_n,mean_relative_iBAQ
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>
1,PB.10027.4:chr12:-|10|1315:112:1117|canonical|ATG,canonical,ATG,PB.10027.4,chr12,-,MATLKEKLIAPVAEEEATVPNNKITVVGVGQVGMACAISILGKSLADELALVDVLEDKLKGEMMDLQHGSLFLQTPKIVADKDYSVTANSKIVVVTAGVRQQEGESRLNLVQRNVNVFKFIIPQIVKYSPDCIIIVVSNPVDILTYVTWKLSGLPKHRVIGSGCNLDSARFRYLMAEKLGIHPSSCHGWILGEHGDSSVAVWSGVNVAGVSLQELNPEMGTDNDSENWKEVHKMVVESAYEVIKLKGYTNWAIGLSVADLIESMLKNLSRIHPVSTMVKGMYGIENEVFLSLPCILNARGLTSVINQKLKDDEVAQLKKSADTLWDIQKDLKDL,334,LDHB,full-splice_match,⋯,359.042,314.479,4163,568.9832,0.3650371,0.06854658,0.2167918,protein_coding,51,0.0007624541


[1] "ORF_id"                      "ORF_type"                   
 [3] "Start_codon"                 "Isoform_id"                 
 [5] "Chr"                         "Strand"                     
 [7] "ORF_seq"                     "ORF_length"                 
 [9] "Geneid"                      "Isoform_structural_category"
[11] "Isoform_subcategory"         "Is_uniprot"                 
[13] "Is_canonical"                "Run_occurrence"             
[15] "RPF_reads"                   "Psites_number"              
[17] "RPF_RPKM"                    "Psites_RPKM"                
[19] "RPF_codon_coverage"          "Psites_codon_coverage"      
[21] "FL"                          "FL_TPM"                     
[23] "N"                           "C"                          
[25] "A"                           "Gene_type"                  
[27] "Unique_peptide_n"            "mean_relative_iBAQ"

Is_canonical,n
<lgl>,<int>
FALSE,34212
TRUE,7853


In [7]:
fread_c("../../results/1/peri/psite_frame_stats.v1.tsv") -> orfs_peri
head(orfs_peri,1)

,ORF_id,CDS_nt_len,n_codons,frame0,frame1,frame2,total_psites,frame0_fraction,frame0_codon_covered
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>
1,PB.1.16:chr1:-|66|1611:871:1024|noncoding|GTG,150,50,0,0,0,0,0,0


In [8]:
fread_c("../../results/1/orf_overlap_inframe.txt") -> orfs_overlap
head(orfs_overlap,1)

,ORF_id,overlap_bp,overlap_fraction,inframe_overlap_bp,inframe_fraction,n_annot_tx_overlapped,n_annot_gene_overlapped,top_hit_gene,top_hit_tx
,<chr>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<chr>,<chr>
1,PB.22711.2:chr19:-|16|1554:181:445|noncoding|ACG,95,0.3639847,0,0,1,1,ENSG00000121410.12,ENST00000263100.8


In [12]:
merge(orfs,orfs_peri) %>% merge(orfs_overlap) -> orfs_1
orfs_1 %>% head(1)
orfs_1 %>% nrow()

,ORF_id,ORF_type,Start_codon,Isoform_id,Chr,Strand,ORF_seq,ORF_length,Geneid,Isoform_structural_category,⋯,frame0_fraction,frame0_codon_covered,overlap_bp,overlap_fraction,inframe_overlap_bp,inframe_fraction,n_annot_tx_overlapped,n_annot_gene_overlapped,top_hit_gene,top_hit_tx
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<chr>,<chr>
1,PB.1.16:chr1:-|66|1611:871:1024|noncoding|GTG,noncoding,GTG,PB.1.16,chr1,-,MLASAPPLPPSTAAPVGQGARQDDSSSSASPSVQGAPREVVDPSGGWPLC,50,WASH7P,novel_not_in_catalog,⋯,0,0,0,0,0,0,0,0,,


[1] 39078

In [14]:
orfs_1 %>% group_by(Is_canonical) %>% summarise(frame0_fraction_mean=mean(frame0_fraction))
orfs_1 %>% group_by(Is_canonical) %>% summarise(inframe_fraction_mean=mean(inframe_fraction))

Is_canonical,frame0_fraction_mean
<lgl>,<dbl>
FALSE,0.1791894
TRUE,0.8328659


Is_canonical,inframe_fraction_mean
<lgl>,<dbl>
FALSE,0.05497943
TRUE,1.00000000


* 需要对orfs进行过滤，过滤掉那些有in-frame overlap的orfs。这部分用来建模。

- 结果（共39078个）
     - 31638 inframe_fraction为0（用于计算进化参数，没有in-frame的部分干扰）
     - 23723 overlap_fraction为0 (适合用于建模，翻译水平以及周期性能算准）
     - 25370 有p sites覆盖
     - 13094 overlap_fraction为0且有PRF reads覆盖（证据较强，没有和任何注释CDS覆盖，有肽段证据且有RPF reads支持）
     - 10653 overlap_fraction为0且有p sites覆盖（证据强，没有和任何注释CDS覆盖，有肽段证据且有p sites支持）
     - 3562 overlap_fraction为0且frame0_fraction大于等于0.5（证据非常强，没有和任何注释CDS覆盖，有肽段证据且三碱基周期性大于等于0.5）

In [26]:
filter(orfs_1,inframe_fraction==0) %>% nrow()
filter(orfs_1,overlap_fraction==0) %>% nrow()
# 等价于第2条
# filter(orfs_1,overlap_fraction==0 & inframe_fraction==0) %>% nrow()
filter(orfs_1,total_psites>0) %>% nrow()
filter(orfs_1,inframe_fraction==0 & overlap_fraction==0 & RPF_reads>0) %>% nrow()
filter(orfs_1,inframe_fraction==0 & overlap_fraction==0 & total_psites>0) %>% nrow()
filter(orfs_1,inframe_fraction==0 & overlap_fraction==0 & total_psites>0 & frame0_fraction>=0.5) %>% nrow()
filter(orfs_1,inframe_fraction==0 & overlap_fraction==0 & total_psites>0) %>% count(ORF_type)
filter(orfs_1,inframe_fraction==0 & overlap_fraction==0 & total_psites>0 & frame0_fraction>=0.5) %>% count(ORF_type)
orfs_1 %>% count(ORF_type)

[1] 31638

[1] 23723

[1] 25370

[1] 10653

[1] 13094

[1] 3562

ORF_type,n
<chr>,<int>
dORF,3619
iORF,4
noncoding,5331
odORF,4
ouORF,7
uORF,1688


ORF_type,n
<chr>,<int>
dORF,1062
noncoding,1834
odORF,3
ouORF,3
uORF,660


ORF_type,n
<chr>,<int>
canonical,4866
dORF,7084
iORF,3287
noncoding,19764
odORF,706
ouORF,743
uORF,2628
